From <a href="https://pytorch.org/tutorials/beginner/chatbot_tutorial.html">Pytorch Tutorial</a> by Matthew Inkawhich

In [1]:
import codecs
import csv
import itertools
import math
import random
import re
import os
import unicodedata
from io import open

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.jit import script, trace

In [2]:
USE_CUDA = torch.cuda.is_available()
device = torch.device('cuda' if USE_CUDA else 'cpu')

# Load and Preprocess Data 
### (Cornell Movie-Dialogues Corpus)

In [3]:
DATA = '../../data'
corpus_name = 'cornell movie-dialogs corpus'
corpus = os.path.join(DATA, corpus_name)

In [4]:
def print_lines(file, n=10):
    with open(file, 'rb') as f:
        lines = f.readlines()
    for line in lines[:n]:
        print(line)

In [5]:
print_lines(os.path.join(corpus, 'movie_lines.txt'))

b'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!\n'
b'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!\n'
b'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.\n'
b'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?\n'
b"L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.\n"
b'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow\n'
b"L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.\n"
b'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No\n'
b'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?\n'
b'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?\n'


### Create Formatted Data File

In [6]:
def load_lines(file_name, fields):
    '''Splits each line of the file into dict of fields'''
    lines = {}
    with open(file_name, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(' +++$+++ ')
            line_obj = {}
            for i, field in enumerate(fields):
                line_obj[field] = values[i]
            lines[line_obj['lineID']] = line_obj
    return lines

In [7]:
def load_conversations(file_name, lines, fields):
    '''
    Groups fields of lines from load_lines() into conversations based on
    movie_conversations.txt
    '''
    conversations = []
    with open(file_name, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(' +++$+++ ')
            conv_obj = {}
            for i, field in enumerate(fields):
                conv_obj[field] = values[i]
            line_ids = eval(conv_obj['utteranceIDs'])
            conv_obj['lines'] = []
            for line_id in line_ids:
                conv_obj['lines'].append(lines[line_id])
            conversations.append(conv_obj)
    return conversations

In [8]:
def extract_sentence_pairs(conversations):
    '''Extract pairs of sentences from conversations'''
    qa_pairs = []
    for conversation in conversations:
        for i in range(len(conversation['lines']) - 1):
            input_line = conversation['lines'][i]['text'].strip()
            target_line = conversation['lines'][i + 1]['text'].strip()
            # Filter if one of the lists is empty
            if input_line and target_line:
                qa_pairs.append([input_line, target_line])
    return qa_pairs

In [9]:
datafile = os.path.join(corpus, 'formatted_movie_lines.txt')
delimiter = '\t'
delimiter = str(codecs.decode(delimiter, 'unicode_escape'))

In [10]:
lines = {}
conversations = []
MOVIE_LINES_FIELDS = [
    'lineID', 'characterID', 'movieID', 'character', 'text']
MOVIE_CONVERSATION_FIELDS = [
    'character1ID', 'character2ID', 'movieID', 'utteranceIDs']

In [11]:
print('\nProcessing corpus...')
lines = load_lines(os.path.join(corpus, 'movie_lines.txt'), 
                   MOVIE_LINES_FIELDS)

print('\nLoading conversations...')
conversations = load_conversations(
    os.path.join(corpus, 'movie_conversations.txt'), 
    lines,
    MOVIE_CONVERSATION_FIELDS)


Processing corpus...

Loading conversations...


In [12]:
print('\nWriting newly formatted file...')
with open(datafile, 'w', encoding='utf-8') as out:
    writer = csv.writer(out, delimiter=delimiter)
    for pair in extract_sentence_pairs(conversations):
        writer.writerow(pair)


Writing newly formatted file...


In [13]:
print('\nSample lines from file')
print_lines(datafile)


Sample lines from file
b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\r\n"
b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\r\n"
b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\r\n"
b"You're asking me out.  That's so cute. What's your name again?\tForget it.\r\n"
b"No, no, it's my fault -- we didn't have a proper introduction ---\tCameron.\r\n"
b"Cameron.\tThe thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\r\n"
b"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\tSeems like she could get a date easy enough...\r\n"

# Load and Trim Data

In [14]:
PAD = 0 # to pad short sentences
SOS = 1 # start of sentence
EOS = 2 # end of sentence

In [15]:
class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD: 'PAD', SOS: 'SOS', EOS: 'EOS'}
        self.n_words = 3 # Count SOS, EOS, PAD
        
    def add_sentence(self, sentence):
        for word in sentence.split():
            self.add_word(word)
            
    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
        else:
            self.word2count[word] += 1
            
    # Remove words below min count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True
        keep_words = []
        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)
        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), 
            len(self.word2index), 
            len(keep_words) / len(self.word2index)))
        
        # Reinit dicts
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD: 'PAD', SOS: 'SOS', EOS: 'EOS'}
        self.n_words = 3
        for word in keep_words:
            self.add_word(word)

In [16]:
MAX_LEN = 10

In [17]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

In [18]:
def normalize_string(s):
    s = unicode_to_ascii(s.lower().strip())
    s = re.sub(r'([.!?])', r' \1', s)
    s = re.sub(r'[^a-zA-Z.!?]+', r' ', s)
    s = re.sub(r'\s+', r' ', s).strip()
    return s

In [19]:
def read_vocs(datafile, corpus_name):
    print('Reading lines...')
    lines = open(datafile, encoding='utf-8').read().strip().split('\n')
    pairs = [[normalize_string(s) for s in line.split('\t')] 
             for line in lines]
    voc = Voc(corpus_name)
    return voc, pairs

In [20]:
def filter_pair(p):
    return len(p[0].split()) < MAX_LEN and len(p[1].split()) < MAX_LEN

In [21]:
def filter_pairs(pairs):
    return [pair for pair in pairs if filter_pair(pair)]

In [22]:
def load_prep_data(corpus, corpus_name, datafile, save_dir):
    print('Start prepping training data...')
    voc, pairs = read_vocs(datafile, corpus_name)
    print('Read {!s} sentence pairs'.format(len(pairs)))
    pairs = filter_pairs(pairs)
    print('Trimmed to {!s} sentence pairs'.format(len(pairs)))
    print('Counting words...')
    for pair in pairs:
        voc.add_sentence(pair[0])
        voc.add_sentence(pair[1])
    print('Counted words:', voc.n_words)
    return voc, pairs

In [23]:
save_dir = '%s/save' % DATA
voc, pairs = load_prep_data(corpus, corpus_name, datafile, save_dir)
print('\npairs:')
for pair in pairs[:10]:
    print(pair)

Start prepping training data...
Reading lines...
Read 221282 sentence pairs
Trimmed to 64271 sentence pairs
Counting words...
Counted words: 3

pairs:
['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']
['hi .', 'looks like things worked out tonight huh ?']
['you know chastity ?', 'i believe we share an art instructor']
['have fun tonight ?', 'tons']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['do you listen to this crap ?', 'what crap ?']
['what good stuff ?', 'the real you .']


In [24]:
MIN_COUNT = 3

In [25]:
def trim_rare_words(voc, pairs):
    voc.trim(MIN_COUNT)
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        for word in input_sentence.split():
            if word not in voc.word2index:
                keep_input = False
                break
        for word in output_sentence.split():
            if word not in voc.word2index:
                keep_output = False
                break
        if keep_input and keep_output:
            keep_pairs.append(pair)
    print('Trimmed from {} pairs to {}, {:.4f} of total'.format(
        len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs

In [26]:
pairs = trim_rare_words(voc, pairs)

keep_words 7822 / 18004 = 0.4345
Trimmed from 64271 pairs to 53165, 0.8272 of total


### Prepare Data for Models

In [36]:
def indices_from_sentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split()] + [EOS]

In [28]:
def zero_padding(l, fill_value=PAD):
    return list(itertools.zip_longest(*l, fillvalue=fill_value))

In [29]:
def binary_matrix(l, value=PAD):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

In [38]:
def input_var(l, voc):
    indices_batch = [indices_from_sentence(voc, sentence) 
                     for sentence in l]
    lengths = torch.tensor([len(indices) for indices in indices_batch])
    pad_list = zero_padding(indices_batch)
    pad_var = torch.LongTensor(pad_list)
    return pad_var, lengths

In [31]:
def output_var(l, voc):
    indices_batch = [indices_from_sentence(voc, sentence) 
                     for sentence in l]
    max_target_len = max([len(indices) for indices in indices_batch])
    pad_list = zero_padding(indices_batch)
    mask = binary_matrix(pad_list)
    mask = torch.ByteTensor(mask)
    pad_var = torch.LongTensor(pad_list)
    return pad_var, mask, max_target_len

In [32]:
def batch2train_data(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split()), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = input_var(input_batch, voc)
    output, mask, max_target_len = output_var(output_batch, voc)
    return inp, lengths, output, mask, max_target_len

In [40]:
# Example for validataion
BATCH = 5
batches = batch2train_data(voc, 
                           [random.choice(pairs) for _ in range(BATCH)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print('input_variable:', input_variable)
print('lengths:', lengths)
print('target_variable:', target_variable)
print('mask:', mask)
print('max_target_len:', max_target_len)

input_variable: tensor([[3, 3, 3, 3, 3],
        [3, 3, 3, 3, 3],
        [3, 3, 3, 2, 2],
        [3, 3, 3, 0, 0],
        [3, 3, 3, 0, 0],
        [3, 2, 2, 0, 0],
        [2, 0, 0, 0, 0]])
lengths: tensor([7, 6, 6, 3, 3])
target_variable: tensor([[3, 3, 3, 3, 3],
        [3, 3, 3, 3, 3],
        [3, 3, 3, 3, 3],
        [3, 3, 3, 3, 3],
        [3, 3, 3, 3, 3],
        [3, 3, 3, 3, 2],
        [3, 3, 3, 2, 0],
        [3, 2, 2, 0, 0],
        [3, 0, 0, 0, 0],
        [2, 0, 0, 0, 0]])
mask: tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0],
        [1, 1, 1, 0, 0],
        [1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0]], dtype=torch.uint8)
max_target_len: 10


# Define Models